In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam

from PIL import Image

In [29]:
# Load the data
split_df = pd.read_csv('split_df.csv', encoding='utf-8-sig')


In [30]:
split_df

,Table 1,Spectacles,Age,Gender,Name,id,Eye,VA,CDR,Diagnosis,IOP
0,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,0,0.6,0.5,0,14.0
1,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,1,0.7,0.9,1,23.0
2,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,0,NaN,NaN,1,25.0
3,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,1,9.0,0.8,1,40.0
4,لا يوجد زرق,0.0,48,1,رندة ناصر,3.0,0,1.0,0.5,0,15.0
...,...,...,...,...,...,...,...,...,...,...,...
85,زرق أيسر تم معالجته جراحيا,1.0,55,1,زريفة المحمد,43.0,1,NaN,NaN,0,NaN
86,مريض زرقي ثانوي بعد استخراج ساد لا يوجد اصابة ...,0.0,71,0,هيثم قزاز,44.0,0,0.3,0.5,1,15.0
87,مريض زرقي ثانوي بعد استخراج ساد لا يوجد اصابة ...,0.0,71,0,هيثم قزاز,44.0,1,0.6,0.7,1,16.0
88,لا يوجد زرق,1.0,65,0,حمود غشام,45.0,0,0.5,0.3,0,15.0


In [31]:
split_df['Age'].value_counts()

Age
64    10
65     6
25     4
67     4
73     4
62     4
55     4
58     4
70     4
50     2
59     2
29     2
17     2
48     2
37     2
52     2
32     2
45     2
60     2
44     2
69     2
43     2
12     2
66     2
24     2
53     2
56     2
39     2
68     2
61     2
76     2
71     2
Name: count, dtype: int64

In [32]:
# age_groups = {
#     'Group_1': [10, 20],
#     'Group_2': [20, 40],
#     'Group_3': [40, 60],
#     'Group_4': [60, 80],
#     'Group_5': [80, 100]
# }

# # Function to encode age into age groups
# def encode_age(age):
#     for group, age_range in age_groups.items():
#         if age_range[0] <= age <= age_range[1]:
#             return group
#     return None

# # Apply encoding to the 'Age' column
# split_df['Age_Group'] = split_df['Age'].apply(encode_age)

# # Perform one-hot encoding
# age_dummies = pd.get_dummies(split_df['Age_Group'], prefix='Age_Group')

# # Concatenate the one-hot encoded age groups with the original DataFrame
# split_df = pd.concat([split_df, age_dummies], axis=1)

# # Drop the original 'Age_Group' column
# split_df.drop(columns=['Age_Group'], inplace=True)

# # Now you have one-hot encoded age groups in your DataFrame
# split_df


,Table 1,Spectacles,Age,Gender,Name,id,Eye,VA,CDR,Diagnosis,IOP,Age_Group_Group_1,Age_Group_Group_2,Age_Group_Group_3,Age_Group_Group_4
0,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,0,0.6,0.5,0,14.0,False,False,False,True
1,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,1,0.7,0.9,1,23.0,False,False,False,True
2,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,0,NaN,NaN,1,25.0,False,False,False,True
3,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,1,9.0,0.8,1,40.0,False,False,False,True
4,لا يوجد زرق,0.0,48,1,رندة ناصر,3.0,0,1.0,0.5,0,15.0,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,زرق أيسر تم معالجته جراحيا,1.0,55,1,زريفة المحمد,43.0,1,NaN,NaN,0,NaN,False,False,True,False
86,مريض زرقي ثانوي بعد استخراج ساد لا يوجد اصابة ...,0.0,71,0,هيثم قزاز,44.0,0,0.3,0.5,1,15.0,False,False,False,True
87,مريض زرقي ثانوي بعد استخراج ساد لا يوجد اصابة ...,0.0,71,0,هيثم قزاز,44.0,1,0.6,0.7,1,16.0,False,False,False,True
88,لا يوجد زرق,1.0,65,0,حمود غشام,45.0,0,0.5,0.3,0,15.0,False,False,False,True


In [33]:
import os

# Assuming you have loaded your schedule data into a DataFrame called 'schedule_df'
# and you have a dictionary mapping names to image filenames called 'name_to_image'

# Example schedule data (replace this with your actual data)
schedule_data = {
    'Name': ['شاهين ميرو',
 'شاهين ميرو',
 'أيمن سمارة',
 'أيمن سمارة',
 'رندة ناصر',
 'رندة ناصر',
 'مصطفى ملا علي',
 'مصطفى ملا علي',
 'نهلة أبو قورة',
 'نهلة أبو قورة',
 'لطيفة إبراهيم',
 'لطيفة إبراهيم',
 'فاطمة فرحان',
 'فاطمة فرحان',
 'صبحي الوزان',
 'صبحي الوزان',
 'هبة لولو',
 'هبة لولو',
 'خديجة كبة',
 'خديجة كبة',
 'عبدالرزاق بويضاني',
 'عبدالرزاق بويضاني',
 'أحمد طحان',
 'أحمد طحان',
 'حليمة شيخو',
 'حليمة شيخو',
 'علي الحسين',
 'علي الحسين',
 'محي الدين خاشوق',
 'محي الدين خاشوق',
 'دلال محمود',
 'دلال محمود',
 'نضال جريج',
 'نضال جريج',
 'فاطمة الخلف',
 'فاطمة الخلف',
 'نجاح النهار',
 'نجاح النهار',
 'مروة محجوب',
 'مروة محجوب',
 'زهير الملاح',
 'زهير الملاح',
 'باسل الباش',
 'باسل الباش',
 'نزار الحلو ',
 'نزار الحلو ',
 'ولاء حماد',
 'ولاء حماد',
 'حسين الحلقي',
 'حسين الحلقي',
 'حصة الحمد',
 'حصة الحمد',
 'ايمان الزعبي',
 'ايمان الزعبي',
 'سلفانة أبو عسلي',
 'سلفانة أبو عسلي',
 'وسام كاشي',
 'وسام كاشي',
 'صالح الهزاع',
 'صالح الهزاع',
 'راما نصرالله',
 'راما نصرالله',
 'أسرار صنوبر',
 'أسرار صنوبر',
 'حنان الابراهيم',
 'حنان الابراهيم',
 'عبدالله الغزولي ',
 'عبدالله الغزولي ',
 'نور عبدالله ',
 'نور عبدالله ',
 'جميلة عبيد',
 'جميلة عبيد',
 'عبدالرحيم حماد',
 'عبدالرحيم حماد',
 'جانيت ضاهر',
 'جانيت ضاهر',
 'فاطمة عوض',
 'فاطمة عوض',
 'حسن القاموع',
 'حسن القاموع',
 'محمد هلال',
 'محمد هلال',
 'عنود الحميد',
 'عنود الحميد',
 'زريفة المحمد',
 'زريفة المحمد',
 'هيثم قزاز',
 'هيثم قزاز',
 'حمود غشام',
 'حمود غشام'],'Eye':[0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1]
    
}
schedule_df = pd.DataFrame(schedule_data)

# Example directory containing images
image_dir = "Data/"

# Example function to load image data
def load_image(filename):
    with open(os.path.join(image_dir, filename), "rb") as f:
        # return f.read()
        return f'{image_dir}{filename}'

# Example dictionary mapping names to image filenames
# You need to populate this dictionary with your actual filenames
name_to_image = {
    ('شاهين ميرو', 0): 'SHAHEN_MERO Optic nerve.png_ left.png',
    ('شاهين ميرو', 1): 'SHAHEN_MERO Optic nerve.png_ right.png',
    ('أيمن سمارة', 1): 'AYMAN_SAMARA_20240121_124719_Disc_3D_R_SINGLE_6mm_512x112.png',
    ('رندة ناصر', 0): 'RAMA_NASR_ALL Optic nerve.png_ left.png',
    ('رندة ناصر', 1): 'RANDA_NASER_2 Optic nerve.png_ right.png',
    ('مصطفى ملا علي', 0): 'MOUSTAFA_MALA_ALI Optic nerve.png_ left.png',
    ('مصطفى ملا علي', 1): 'MOUSTAFA_MALA_ALI Optic nerve.png_ right.png',
    ('نهلة أبو قورة', 0): '6361_NAHLA_ABO_QORA_left.png',
    ('لطيفة إبراهيم', 0): 'LATEFA_IBRAHEM Optic nerve.png_ left.png',
    ('لطيفة إبراهيم', 1): 'LATEFA_IBRAHEM Optic nerve.png_ right.png',
    ('فاطمة فرحان', 0): 'FATEMA_FARHAN Optic nerve.png_ left.png',
    ('فاطمة فرحان', 1): 'FATEMA_FARHAN Optic nerve.png_ right.png',
    ('صبحي الوزان', 0): 'SOBHI_WAZAN Optic nerve.png_ left.png',
    ('صبحي الوزان', 1): 'SOBHI_WAZAN Optic nerve.png_ right.png',
    ('هبة لولو', 0): 'HIBA_LOLO Optic nerve.png_ left.png',
    ('هبة لولو', 1): 'HIBA_LOLO Optic nerve.png_ right.png',
    ('خديجة كبة', 0): 'KHADEGA_KABEH Optic nerve.png_ left.png',
    ('خديجة كبة', 1): 'KHADEGA_KABEH Optic nerve.png_ right.png',
    ('عبدالرزاق بويضاني', 0): 'ABD_ALRAZAQ_BOEDANI Optic nerve.png_ left.png',
    ('عبدالرزاق بويضاني', 1): 'ABD_ALRAZAQ_BOEDANI Optic nerve.png_ right.png',
    ('أحمد طحان', 0): 'AHMAD_TAHAN Optic nerve.png_ left.png',
    ('أحمد طحان', 1): 'AHMAD_TAHAN Optic nerve.png_ right.png',
    ('حليمة شيخو', 0): '5919_HALEMA_SHEKHO Optic nerve.png_ left.png',
    ('حليمة شيخو', 1): '5919_HALEMA_SHEKHO Optic nerve.png_ right.png',
    ('علي الحسين', 0): 'ALI_HOSEN Optic nerve.png_ left.png',
    ('علي الحسين', 1): 'ALI_HOSEN Optic nerve.png_ right.png',
    ('محي الدين خاشوق', 0): 'MOHY_ALDEN_KHASHOQ Optic nerve.png_ left.png',
    ('محي الدين خاشوق', 1): 'MOHY_ALDEN_KHASHOQ Optic nerve.png_ right.png',
    ('دلال محمود', 0): 'DALAL_MAHMODOptic nerve.png_ left.png',
    ('دلال محمود', 1): 'DALAL_MAHMODOptic nerve.png_ right.png',
    ('نضال جريج', 0): 'NEDAL_JREJ Optic nerve.png_ left.png',
    ('نضال جريج', 1): 'NEDAL_JREJ Optic nerve.png_ right.png',
    ('فاطمة الخلف', 0): 'FATEMAH_KHALAF Optic nerve.png_ left.png',
    ('فاطمة الخلف', 1): 'FATEMAH_KHALAF Optic nerve.png_ right.png',
    ('نجاح النهار', 0): 'NAJAH_NAHAR  Optic nerve.png_ left.png',
    ('نجاح النهار', 1): 'NAJAH_NAHAR  Optic nerve.png_ right.png',
    ('مروة محجوب', 0): '5941_MARWA_MAHJOB Optic nerve.png_ left.png',
    ('مروة محجوب', 1): '5941_MARWA_MAHJOB Optic nerve.png_ right.png',
    ('زهير الملاح', 0): 'ZOHER_MALAH Optic nerve.png_ left.png',
    ('زهير الملاح', 1): 'ZOHER_MALAH Optic nerve.png_ right.png',
    ('باسل الباش', 0): 'BASEL_BASH Optic nerve.png_ left.png',
    ('باسل الباش', 1): 'BASEL_BASH Optic nerve.png_ right.png',
    ('نزار الحلو ', 0): 'NEZAR_HELOU Optic nerve.png_ left.png',
    ('نزار الحلو ', 1): 'NEZAR_HELOU Optic nerve.png_ right.png',
    ('ولاء حماد', 0): 'WALAA_HAMAD Optic nerve.png_ left.png',
    ('ولاء حماد', 1): 'WALAA_HAMAD Optic nerve.png_ right.png',
    ('حسين الحلقي', 0): 'HOSEN_HALAQI Optic nerve.png_ left.png',
    ('حسين الحلقي', 1): 'HOSEN_HALAQI Optic nerve.png_ right.png',
    ('حصة الحمد', 1): 'HASA_HAMAD_20240125_100552_Disc_3D_R_SINGLE_6mm_512x112_right.png',
    ('سلفانة أبو عسلي', 0): 'SELVANA_ABO_AS Optic nerve.png_ left.png',
    ('سلفانة أبو عسلي', 1): 'SELVANA_ABO_AS Optic nerve.png_ right.png',
    ('وسام كاشي', 0): 'WESAM_KASHI Optic nerve.png_ left.png',
    ('وسام كاشي', 1): 'WESAM_KASHI Optic nerve.png_ right.png',
    ('صالح الهزاع', 0): 'SALEH_HAZAA_20 Optic nerve.png_ left.png',
    ('صالح الهزاع', 1): 'SALEH_HAZAA_20 Optic nerve.png_ right.png',
    ('راما نصرالله', 0): 'RAMA_NASR_ALL Optic nerve.png_ left.png',
    ('راما نصرالله', 1): 'RAMA_NASR_ALL Optic nerve.png_ right.png',
    ('حنان الابراهيم', 0): 'HANAN_IBRAHEM Optic nerve.png_ left.png',
    ('حنان الابراهيم', 1): 'HANAN_IBRAHEM Optic nerve.png_ right.png',
    ('عبدالله الغزولي ', 0): 'ABD_ALLAH_GHAZOLI_20231007_11 Optic nerve.png_ left.png',
    ('عبدالله الغزولي ', 1): 'ABD_ALLAH_GHAZOLI_20231007_11 Optic nerve.png_ right.png',
    ('نور عبدالله ', 0): 'NOUR_ABD_ALLAH  Optic nerve.png_ left.png',
    ('نور عبدالله ', 1): 'NOUR_ABD_ALLAH  Optic nerve.png_ right.png',
    ('جميلة عبيد', 0): 'JAMELEH_ABED Optic nerve.png_ left.png',
    ('جميلة عبيد', 1): 'JAMELEH_ABED Optic nerve.png_ right.png',
    ('عبدالرحيم حماد', 0): 'ABD_ALRAHEM_HAMAD Optic nerve.png_ left.png',
    ('عبدالرحيم حماد', 1): 'ABD_ALRAHEM_HAMAD Optic nerve.png_ right.png',
    ('جانيت ضاهر', 0): 'JANET_DAHER Optic nerve.png_ left.png',
    ('جانيت ضاهر', 1): 'JANET_DAHER Optic nerve.png_ right.png',
    ('فاطمة عوض', 0): 'FATEMA_AWAD_20 Optic nerve.png_ left.png',
    ('فاطمة عوض', 1): 'FATEMA_AWAD_20 Optic nerve.png_ right.png',
    ('حسن القاموع', 0): 'HASAN_QAMOAAOptic nerve.png_ left.png',
    ('حسن القاموع', 1): 'HASAN_QAMOAAOptic nerve.png_ right.png',
    ('محمد هلال', 0): 'MOUHAMAD_HELAL_ Optic nerve.png_ left.png',
    ('محمد هلال', 1): 'MOUHAMAD_HELAL_ Optic nerve.png_ left.png',
    ('زريفة المحمد', 0): 'ZAREFA_MOUHAMAD_20231227_103707_Disc_3D_L_SINGLE_6mm_512x112_left.png',
    ('هيثم قزاز', 0): 'HAETHAM_QAZAZ Optic nerve.png_ left.png',
    ('هيثم قزاز', 1): 'HAETHAM_QAZAZ Optic nerve.png_ right.png',
}

# Function to get the image data based on the name
def get_image_data(name):
    filename = name_to_image.get(name ,None)
    if filename:
        return load_image(filename)
    else:
        return None

# Add a new column 'Image' to the DataFrame and fill it with image data
split_df['Image'] = schedule_df.apply(lambda row: get_image_data((row['Name'], row['Eye'])), axis=1)

# Display the DataFrame with image data
split_df.head(17)


,Table 1,Spectacles,Age,Gender,Name,id,Eye,VA,CDR,Diagnosis,IOP,Age_Group_Group_1,Age_Group_Group_2,Age_Group_Group_3,Age_Group_Group_4,Image
0,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,0,0.6,0.5,0,14.0,False,False,False,True,Data/SHAHEN_MERO Optic nerve.png_ left.png
1,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,1,0.7,0.9,1,23.0,False,False,False,True,Data/SHAHEN_MERO Optic nerve.png_ right.png
2,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,0,NaN,NaN,1,25.0,False,False,False,True,None
3,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,1,9.0,0.8,1,40.0,False,False,False,True,Data/AYMAN_SAMARA_20240121_124719_Disc_3D_R_SI...
4,لا يوجد زرق,0.0,48,1,رندة ناصر,3.0,0,1.0,0.5,0,15.0,False,False,True,False,Data/RAMA_NASR_ALL Optic nerve.png_ left.png
5,لا يوجد زرق,0.0,48,1,رندة ناصر,3.0,1,1.0,0.4,0,14.0,False,False,True,False,Data/RANDA_NASER_2 Optic nerve.png_ right.png
6,زرق مزدوج أشد بالأيمن,1.0,60,0,مصطفى ملا علي,4.0,0,0.2,0.7,1,10.0,False,False,True,False,Data/MOUSTAFA_MALA_ALI Optic nerve.png_ left.png
7,زرق مزدوج أشد بالأيمن,1.0,60,0,مصطفى ملا علي,4.0,1,0.1,0.7,1,14.0,False,False,True,False,Data/MOUSTAFA_MALA_ALI Optic nerve.png_ right.png
8,زرق بالعينتين أشد بالأيمن,1.0,58,1,نهلة أبو قورة,5.0,0,0.5,NaN,1,8.0,False,False,True,False,Data/6361_NAHLA_ABO_QORA_left.png
9,زرق بالعينتين أشد بالأيمن,1.0,58,1,نهلة أبو قورة,5.0,1,1.0,NaN,1,11.0,False,False,True,False,None


In [34]:
split_df = split_df.drop(['Table 1', 'Gender', 'id', 'Age', 'Spectacles'], axis=1)

In [35]:
split_df['Image'].isnull().sum()

12

In [36]:
# split_df['Image']
split_df[split_df['Image'].isnull()]

,Name,Eye,VA,CDR,Diagnosis,IOP,Age_Group_Group_1,Age_Group_Group_2,Age_Group_Group_3,Age_Group_Group_4,Image
2,أيمن سمارة,0,NaN,NaN,1,25.0,False,False,False,True,None
9,نهلة أبو قورة,1,1.0,NaN,1,11.0,False,False,True,False,None
50,حصة الحمد,0,0.5,0.3,0,NaN,False,False,False,True,None
52,ايمان الزعبي,0,1.0,0.3,0,18.0,False,False,True,False,None
53,ايمان الزعبي,1,1.0,0.3,0,19.0,False,False,True,False,None
62,أسرار صنوبر,0,1.0,NaN,0,18.0,False,False,True,False,None
63,أسرار صنوبر,1,9.0,NaN,0,18.0,False,False,True,False,None
82,عنود الحميد,0,0.4,0.3,0,18.0,False,False,False,True,None
83,عنود الحميد,1,0.5,0.3,0,17.0,False,False,False,True,None
85,زريفة المحمد,1,NaN,NaN,0,NaN,False,False,True,False,None


In [37]:
split_df = split_df.drop(['Name','Eye'], axis=1)

In [38]:
split_df.dropna(inplace=True)

In [39]:
split_df

,VA,CDR,Diagnosis,IOP,Age_Group_Group_1,Age_Group_Group_2,Age_Group_Group_3,Age_Group_Group_4,Image
0,0.6,0.5,0,14.0,False,False,False,True,Data/SHAHEN_MERO Optic nerve.png_ left.png
1,0.7,0.9,1,23.0,False,False,False,True,Data/SHAHEN_MERO Optic nerve.png_ right.png
3,9.0,0.8,1,40.0,False,False,False,True,Data/AYMAN_SAMARA_20240121_124719_Disc_3D_R_SI...
4,1.0,0.5,0,15.0,False,False,True,False,Data/RAMA_NASR_ALL Optic nerve.png_ left.png
5,1.0,0.4,0,14.0,False,False,True,False,Data/RANDA_NASER_2 Optic nerve.png_ right.png
...,...,...,...,...,...,...,...,...,...
80,0.7,0.8,0,13.0,False,True,False,False,Data/MOUHAMAD_HELAL_ Optic nerve.png_ left.png
81,9.0,0.8,1,16.0,False,True,False,False,Data/MOUHAMAD_HELAL_ Optic nerve.png_ left.png
84,0.2,0.8,1,10.0,False,False,True,False,Data/ZAREFA_MOUHAMAD_20231227_103707_Disc_3D_L...
86,0.3,0.5,1,15.0,False,False,False,True,Data/HAETHAM_QAZAZ Optic nerve.png_ left.png


In [11]:
split_df.count()

VA           64
CDR          64
Diagnosis    64
IOP          64
Image        64
dtype: int64

In [12]:
# Load and preprocess images for each patient
image_paths =  split_df['Image'].tolist()


In [13]:
image_paths

['Data/SHAHEN_MERO Optic nerve.png_ left.png',
 'Data/SHAHEN_MERO Optic nerve.png_ right.png',
 'Data/AYMAN_SAMARA_20240121_124719_Disc_3D_R_SINGLE_6mm_512x112.png',
 'Data/RAMA_NASR_ALL Optic nerve.png_ left.png',
 'Data/RANDA_NASER_2 Optic nerve.png_ right.png',
 'Data/MOUSTAFA_MALA_ALI Optic nerve.png_ left.png',
 'Data/MOUSTAFA_MALA_ALI Optic nerve.png_ right.png',
 'Data/LATEFA_IBRAHEM Optic nerve.png_ left.png',
 'Data/LATEFA_IBRAHEM Optic nerve.png_ right.png',
 'Data/FATEMA_FARHAN Optic nerve.png_ left.png',
 'Data/FATEMA_FARHAN Optic nerve.png_ right.png',
 'Data/SOBHI_WAZAN Optic nerve.png_ left.png',
 'Data/SOBHI_WAZAN Optic nerve.png_ right.png',
 'Data/HIBA_LOLO Optic nerve.png_ left.png',
 'Data/HIBA_LOLO Optic nerve.png_ right.png',
 'Data/KHADEGA_KABEH Optic nerve.png_ left.png',
 'Data/KHADEGA_KABEH Optic nerve.png_ right.png',
 'Data/ABD_ALRAZAQ_BOEDANI Optic nerve.png_ left.png',
 'Data/ABD_ALRAZAQ_BOEDANI Optic nerve.png_ right.png',
 'Data/5919_HALEMA_SHEKHO Optic 

In [14]:
images = [np.array(Image.open(path).convert('RGB').resize((200, 200))) for path in image_paths]  # Load and resize images
X_images = np.array(images) / 255.0  # Normalize pixel values to range [0, 1]

In [15]:
# Drop columns not used for prediction
y_data = split_df['Diagnosis']
X_data = split_df.drop(['Image','Diagnosis'], axis=1)

In [16]:
# Split data into train and test sets
X_train, X_test, X_images_train, X_images_test, y_train, y_test = train_test_split(X_data, X_images, y_data, test_size=0.1, random_state=44, shuffle=True)


In [17]:
# Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D((2, 2)),
    Dropout(rate=0.2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid') 
])

In [18]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 99, 99, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 99, 99, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 48, 48, 64)        0         
 g2D)                                                            


                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 23, 23, 128)       0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 23, 23, 128)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 21, 21, 128)       147584    
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 10, 10, 128)       0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 12800)             0         
                                                                 
 dense (De

In [20]:
# Train the model
history = model.fit(X_images_train, y_train, epochs=20, batch_size=32, validation_data=(X_images_test, y_test))


Epoch 1/20


2/2 [==============================] - 11s 2s/step - loss: 1.8938 - accuracy: 0.4211 - val_loss: 0.6911 - val_accuracy: 0.5714
Epoch 2/20
2/2 [==============================] - 3s 1s/step - loss: 0.7266 - accuracy: 0.4737 - val_loss: 0.6994 - val_accuracy: 0.4286
Epoch 3/20
2/2 [==============================] - 2s 990ms/step - loss: 0.7094 - accuracy: 0.3684 - val_loss: 0.6936 - val_accuracy: 0.4286
Epoch 4/20
2/2 [==============================] - 2s 1s/step - loss: 0.6841 - accuracy: 0.5614 - val_loss: 0.6890 - val_accuracy: 0.5714
Epoch 5/20
2/2 [==============================] - 2s 1s/step - loss: 0.6592 - accuracy: 0.6316 - val_loss: 0.6937 - val_accuracy: 0.5714
Epoch 6/20
2/2 [==============================] - 3s 1s/step - loss: 0.6764 - accuracy: 0.6316 - val_loss: 0.6890 - val_accuracy: 0.5714
Epoch 7/20
2/2 [==============================] - 3s 1s/step - loss: 0.6436 - accuracy: 0.6316 - val_loss: 0.6904 - val_accuracy: 0.5714
Epoch 8/20
2/2 [===================

KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = model.evaluate(X_images_test, y_test)
print(f"Test Accuracy: {test_acc}")

1/1 [==============================] - 0s 111ms/step - loss: 0.9338 - accuracy: 0.5714
Test Accuracy: 0.5714285969734192


In [ ]:
# Evaluate the model
train_score = model.evaluate(X_images_train, y_train)
test_score = model.evaluate(X_images_test, y_test)
print('CNN Train Score:', train_score)
print('CNN Test Score:', test_score)


1/1 [==============================] - 0s 119ms/step - loss: 0.9338 - accuracy: 0.5714
CNN Train Score: [0.34277939796447754, 0.8245614171028137]
CNN Test Score: [0.9337977170944214, 0.5714285969734192]


In [ ]:
# Make predictions on the test set
predictions = model.predict(X_images_test)


1/1 [==============================] - 0s 284ms/step


In [ ]:
# # Generate classification report and confusion matrix
# print('Classification Report:')
# print(classification_report(y_test, predictions))

# print('Confusion Matrix:')
# print(confusion_matrix(y_test, predictions))


In [ ]:
from keras.backend import clear_session
clear_session()